On a obtenu une base de donnée regroupant des infos sur des séries des années 2010-2019. Cependant, certaines de ces infos ne sont pas tout de suite exploitable, soit parcequ'elles sont sous un mauvais format, soit parcequ'il y a des éléments parasites (ex: dans le genre d'une série, les "et" et virgules peuvent être gênants si on veut étudier leur influence sur la note) 

In [1]:
import bs4
import pandas as pd
from urllib import request
import re

In [2]:
series_info=pd.read_csv('C:/Users/Jean-Baptiste/OneDrive/ENSAE/projet_python_data.csv')
#à cause du passage sous csv, le nom n'est plus la clef et quand on importe le fichier, la colonne se retrouve nommée "Unnamed: 0", on change donc le nom
series_info.rename(columns = {'Unnamed: 0':'Nom'}, inplace = True) 
series_info

,Nom,Note,Auteur,Genre,Nombre de saisons,Plateforme,Durée,Date de sortie
0,Sherlock,8.1,Série de Mark Gatiss et Steven Moffat,"Policier, drame et thriller",5 saisons(en cours),BBC One,1 h 30 min,25 juillet 2010
1,The Walking Dead,6.7,Série de Frank Darabont,Épouvante-horreur et drame,10 saisons(en cours),AMC,43 min,11 octobre 2010
2,Boardwalk Empire,7.6,Série de Terence Winter,Drame,5 saisons(terminée),HBO,1 h,19 septembre 2010
3,Luther,7.6,Série de Neil Cross,Policier et drame,5 saisons(en cours),BBC One,1 h,4 mai 2010
4,Band of Brothers : L'enfer du Pacifique,7.5,Série de Bruce McKenna,"Mini-série, drame et guerre",1 saison(terminée),HBO,50 min,14 mars 2010
5,Downton Abbey,7.6,Série de Julian Fellowes,Drame,6 saisons(terminée),ITV,1 h,26 septembre 2010
6,Spartacus,6.7,Série de Steven S. DeKnight,"Action, drame et historique",3 saisons(terminée),Starz,1 h,22 janvier 2010
7,Game of Thrones,8.2,Série de David Benioff et D.B. Weiss,"Aventure, drame et fantasy",8 saisons(terminée),HBO,52 min,5 décembre 2010
8,Treme,8.1,Série de David Simon et Eric Overmyer,Drame,4 saisons(terminée),HBO,1 h,11 avril 2010
9,Justified,7.2,Série de Graham Yost,"Policier, drame et action",6 saisons(terminée),FX,45 min,16 mars 2010


On s'apperçoit d'un problème, la plateforme de diffusion n'étant pas toujours indiquée, il peut y avoir un décalage vers la gauche des dernières colonnes: la plateforme contient la durée, la durée la date de sortie et la date contient ['Inconnu'] (cf les 2 exemples ci dessous), on va donc devoir redécaler ces colonnes vers la droite.

In [3]:
series_info[series_info.Nom == 'Black Lagoon: Roberta`s Blood Trail']

,Nom,Note,Auteur,Genre,Nombre de saisons,Plateforme,Durée,Date de sortie
34,Black Lagoon: Roberta`s Blood Trail,7.3,"Anime (OAV) de Rei Hiroe, Shino Masanori e...","Drame, action et animation",1 saison(terminée),33 min,27 juin 2010,Inconnu


In [4]:
series_info[series_info.Nom == 'Julfou']

,Nom,Note,Auteur,Genre,Nombre de saisons,Plateforme,Durée,Date de sortie
163,Julfou,6.5,Websérie de Julien Josselin (Julfou),Comédie et mini-série,3 min,12 juillet 2011,Inconnu,Inconnu


In [5]:
#pour identifier une telle ligne, il suffit que h ou min se trouve parmi les mots de la plateforme 
for k in range(len(series_info)):
    if 'h' in series_info.loc[k,'Plateforme'].split() or 'min' in series_info.loc[k,'Plateforme'].split():
        series_info.loc[k,'Date de sortie'] = series_info.loc[k,'Durée']
        series_info.loc[k,'Durée'] = series_info.loc[k,'Plateforme']
        series_info.loc[k,'Plateforme'] = 'Inconnue'
    if 'h' in series_info.loc[k,'Nombre de saisons'].split() or 'min' in series_info.loc[k,'Nombre de saisons'].split():
        series_info.loc[k,'Date de sortie'] = series_info.loc[k,'Plateforme']
        series_info.loc[k,'Durée'] = series_info.loc[k,'Nombre de saisons']
        series_info.loc[k,'Plateforme'] = 'Inconnue'
        series_info.loc[k,'Nombre de saisons'] = 'Inconnue'

On obtient des formats satisfaisants

In [6]:
series_info[series_info.Nom == 'Black Lagoon: Roberta`s Blood Trail']

,Nom,Note,Auteur,Genre,Nombre de saisons,Plateforme,Durée,Date de sortie
34,Black Lagoon: Roberta`s Blood Trail,7.3,"Anime (OAV) de Rei Hiroe, Shino Masanori e...","Drame, action et animation",1 saison(terminée),Inconnue,33 min,27 juin 2010


In [7]:
series_info[series_info.Nom == 'Julfou']

,Nom,Note,Auteur,Genre,Nombre de saisons,Plateforme,Durée,Date de sortie
163,Julfou,6.5,Websérie de Julien Josselin (Julfou),Comédie et mini-série,Inconnue,Inconnue,3 min,12 juillet 2011


In [8]:
series_info.loc[169,'Date de sortie']=series_info.loc[169,'Plateforme']
series_info.loc[169,'Plateforme']='Inconnue'

# I - La date de sortie & la durée

En soi la date exacte de sortie n'est pas très importante. On peut cependant s'intéresser à l'année ainsi qu'au mois de sortie: le calendrier de sortie se basant sur les saisons, on peut trouver intéressant de savoir si une série sortant au printemps est mieux appéciée qu'une sortant en automne.

In [10]:
#on transforme la date en une liste avec le numéro du jour, le mois et l'année
series_info1=series_info.copy()
series_info1['Date de sortie']=series_info['Date de sortie'].apply(lambda x: x.split())
def extract_month(l):
    if len(l) == 3:
        return(l[1])
    else:
        return(l[0])
def extract_year(l):
    if len(l) == 3:
        return(l[2])
    else:
        return(l[1])
series_info1['Mois']=series_info1['Date de sortie'].apply(extract_month)
series_info1['Année']=series_info1['Date de sortie'].apply(extract_year)


In [11]:
series_info1.head()

,Nom,Note,Auteur,Genre,Nombre de saisons,Plateforme,Durée,Date de sortie,Mois,Année
0,Sherlock,8.1,Série de Mark Gatiss et Steven Moffat,"Policier, drame et thriller",5 saisons(en cours),BBC One,1 h 30 min,"[25, juillet, 2010]",juillet,2010
1,The Walking Dead,6.7,Série de Frank Darabont,Épouvante-horreur et drame,10 saisons(en cours),AMC,43 min,"[11, octobre, 2010]",octobre,2010
2,Boardwalk Empire,7.6,Série de Terence Winter,Drame,5 saisons(terminée),HBO,1 h,"[19, septembre, 2010]",septembre,2010
3,Luther,7.6,Série de Neil Cross,Policier et drame,5 saisons(en cours),BBC One,1 h,"[4, mai, 2010]",mai,2010
4,Band of Brothers : L'enfer du Pacifique,7.5,Série de Bruce McKenna,"Mini-série, drame et guerre",1 saison(terminée),HBO,50 min,"[14, mars, 2010]",mars,2010


Idem pour la durée, on la préfère en minute plutôt qu'en heure & minutes

In [12]:
series_info1['Durée']=series_info1['Durée'].apply(lambda x: x.split())

In [13]:
series_info1.head()

,Nom,Note,Auteur,Genre,Nombre de saisons,Plateforme,Durée,Date de sortie,Mois,Année
0,Sherlock,8.1,Série de Mark Gatiss et Steven Moffat,"Policier, drame et thriller",5 saisons(en cours),BBC One,"[1, h, 30, min]","[25, juillet, 2010]",juillet,2010
1,The Walking Dead,6.7,Série de Frank Darabont,Épouvante-horreur et drame,10 saisons(en cours),AMC,"[43, min]","[11, octobre, 2010]",octobre,2010
2,Boardwalk Empire,7.6,Série de Terence Winter,Drame,5 saisons(terminée),HBO,"[1, h]","[19, septembre, 2010]",septembre,2010
3,Luther,7.6,Série de Neil Cross,Policier et drame,5 saisons(en cours),BBC One,"[1, h]","[4, mai, 2010]",mai,2010
4,Band of Brothers : L'enfer du Pacifique,7.5,Série de Bruce McKenna,"Mini-série, drame et guerre",1 saison(terminée),HBO,"[50, min]","[14, mars, 2010]",mars,2010


In [14]:
def convertion_minutes(l):
    if len(l)==1:
        return('Inconnue')
    elif len(l)==2:
        if 'min' in l:
            return (float(l[0]))
        else:
            return(60*float(l[0]))
    else:
        return (float(l[0])*60+float(l[2]))

In [15]:
series_info2=series_info1.copy()
series_info2['Durée']=series_info2['Durée'].apply(convertion_minutes)

In [29]:
series_info2.head()

,Nom,Note,Auteur,Genre,Nombre de saisons,Plateforme,Durée,Date de sortie,Mois,Année
0,Sherlock,8.1,Série de Mark Gatiss et Steven Moffat,"Policier, drame et thriller",5 saisons(en cours),BBC One,90,"[25, juillet, 2010]",juillet,2010
1,The Walking Dead,6.7,Série de Frank Darabont,Épouvante-horreur et drame,10 saisons(en cours),AMC,43,"[11, octobre, 2010]",octobre,2010
2,Boardwalk Empire,7.6,Série de Terence Winter,Drame,5 saisons(terminée),HBO,60,"[19, septembre, 2010]",septembre,2010
3,Luther,7.6,Série de Neil Cross,Policier et drame,5 saisons(en cours),BBC One,60,"[4, mai, 2010]",mai,2010
4,Band of Brothers : L'enfer du Pacifique,7.5,Série de Bruce McKenna,"Mini-série, drame et guerre",1 saison(terminée),HBO,50,"[14, mars, 2010]",mars,2010


# II - Nombre de saisons


On préfère garder le nombre de saisons pour la suite au lieu d'avoir les précisions de 'en cours' ou 'terminé'

In [17]:
def nb_saisons(l):
    try:
        return(float(l.split()[0]))
    except ValueError : 
        return ('Inconnu')

In [18]:
series_info3=series_info2.copy()
series_info3['Nombre de saisons']=series_info3['Nombre de saisons'].apply(nb_saisons)

In [28]:
series_info3.head()

,Nom,Note,Auteur,Genre,Nombre de saisons,Plateforme,Durée,Date de sortie,Mois,Année
0,Sherlock,8.1,Série de Mark Gatiss et Steven Moffat,"Policier, drame et thriller",5,BBC One,90,"[25, juillet, 2010]",juillet,2010
1,The Walking Dead,6.7,Série de Frank Darabont,Épouvante-horreur et drame,10,AMC,43,"[11, octobre, 2010]",octobre,2010
2,Boardwalk Empire,7.6,Série de Terence Winter,Drame,5,HBO,60,"[19, septembre, 2010]",septembre,2010
3,Luther,7.6,Série de Neil Cross,Policier et drame,5,BBC One,60,"[4, mai, 2010]",mai,2010
4,Band of Brothers : L'enfer du Pacifique,7.5,Série de Bruce McKenna,"Mini-série, drame et guerre",1,HBO,50,"[14, mars, 2010]",mars,2010


# III - Genre & Plateforme

Pour la suite et rendre ces critères plus faciles à manipuler, on va transformer le critère 'genre' en une liste de genres, on va donc devoir éliminer la ponctuations, les 'et' et séparer les mots

In [20]:
series_info4=series_info3.copy()
def espace(l):
    l=l.replace(',','')
    return(l)
def stepwords(l):
    if 'et' in l:
        l.remove('et')
    return(l)
#on met les lettres des genres en minuscule
series_info4['Genre']=series_info4['Genre'].apply(lambda x: espace(x))
series_info4['Genre']=series_info4['Genre'].apply(lambda x: x.lower())
series_info4['Genre']=series_info4['Genre'].apply(lambda x: x.split())
series_info4['Genre']=series_info4['Genre'].apply(stepwords)


In [30]:
series_info4.head()

,Nom,Note,Auteur,Genre,Nombre de saisons,Plateforme,Durée,Date de sortie,Mois,Année
0,Sherlock,8.1,Série de Mark Gatiss et Steven Moffat,"[policier, drame, thriller]",5,BBC One,90,"[25, juillet, 2010]",juillet,2010
1,The Walking Dead,6.7,Série de Frank Darabont,"[épouvante-horreur, drame]",10,AMC,43,"[11, octobre, 2010]",octobre,2010
2,Boardwalk Empire,7.6,Série de Terence Winter,[drame],5,HBO,60,"[19, septembre, 2010]",septembre,2010
3,Luther,7.6,Série de Neil Cross,"[policier, drame]",5,BBC One,60,"[4, mai, 2010]",mai,2010
4,Band of Brothers : L'enfer du Pacifique,7.5,Série de Bruce McKenna,"[mini-série, drame, guerre]",1,HBO,50,"[14, mars, 2010]",mars,2010


On cherche donc à voir quels sont les différents genres qui peuvent qualifier une série

In [22]:
genres=[]
for k in range(len(series_info4)):
    for l in series_info4.loc[k,'Genre']:
        if not l in genres:
            genres.append(l)

In [23]:
genres

['policier',
 'drame',
 'thriller',
 'épouvante-horreur',
 'mini-série',
 'guerre',
 'action',
 'historique',
 'aventure',
 'fantasy',
 'comédie',
 'animation',
 'jeunesse',
 'documentaire',
 'dramatique',
 'romance',
 'shōnen',
 'seinen',
 'fantastique',
 'science-fiction',
 'sport',
 'biopic',
 'musique',
 'anthologique',
 'judiciaire',
 'western',
 'médical',
 'télé-réalité']

In [24]:
series_info4['Plateforme'].unique()

array(['BBC One', 'AMC', 'HBO', 'ITV', 'Starz', 'FX', 'FX (US)', 'DR1',
       'DC Universe', 'Fuji TV', 'Allociné', 'Channel 4', 'Direct 8',
       'FOX (US)', 'Comedy Central (US)', 'TNT (US)', 'Cartoon Network',
       'Cinemax', 'SVT', 'Canal+', 'Freeform', 'CBS', 'Disney XD',
       'NTV (JP)', 'AnimeDigitalNetwork', 'Showtime', 'The CW',
       'Inconnue', 'Tokyo Broadcasting System', 'NBC', 'YouTube', 'SyFy',
       'TV Tokyo', 'Paramount Network', 'Tokyo MX', 'BS11',
       'Canal + Family', 'Discovery Family', 'BBC Three', 'TF1', 'Animax',
       'Keshet', 'SBS (KR)', 'Gulli', 'ABC (US)', 'Arte', 'MTV (US)',
       'TV Land', 'téva', 'MBS', 'Global', 'Sky1', 'A&E', 'CBeebies',
       'KBS TV2', 'Teletoon', 'Showcase (CA)', 'CTC (JA)', 'RTÉ One',
       'Netflix', 'USA Network', 'City', 'BBC Two', 'France 2',
       'jeuxvideo.com', 'ABC (AU)', 'Sky Atlantic (IT)', 'France 5', 'W9',
       'Antena 3', 'tvk', 'TV Aichi', 'PBS', 'Hunan TV', 'TV3 (ES)',
       'ReelzChannel', 'Nic

Les plateformes étant déjà dans un format satisfaisant, on va juste éliminer le doublon FX/FX(US) qui correspond à la même plateforme

In [25]:
for k in range(len(series_info4)):
    if series_info4.loc[k,'Plateforme']=='FX (US)':
        series_info4.loc[k,'Plateforme']='FX'

In [26]:
series_info4['Plateforme'].unique()

array(['BBC One', 'AMC', 'HBO', 'ITV', 'Starz', 'FX', 'DR1',
       'DC Universe', 'Fuji TV', 'Allociné', 'Channel 4', 'Direct 8',
       'FOX (US)', 'Comedy Central (US)', 'TNT (US)', 'Cartoon Network',
       'Cinemax', 'SVT', 'Canal+', 'Freeform', 'CBS', 'Disney XD',
       'NTV (JP)', 'AnimeDigitalNetwork', 'Showtime', 'The CW',
       'Inconnue', 'Tokyo Broadcasting System', 'NBC', 'YouTube', 'SyFy',
       'TV Tokyo', 'Paramount Network', 'Tokyo MX', 'BS11',
       'Canal + Family', 'Discovery Family', 'BBC Three', 'TF1', 'Animax',
       'Keshet', 'SBS (KR)', 'Gulli', 'ABC (US)', 'Arte', 'MTV (US)',
       'TV Land', 'téva', 'MBS', 'Global', 'Sky1', 'A&E', 'CBeebies',
       'KBS TV2', 'Teletoon', 'Showcase (CA)', 'CTC (JA)', 'RTÉ One',
       'Netflix', 'USA Network', 'City', 'BBC Two', 'France 2',
       'jeuxvideo.com', 'ABC (AU)', 'Sky Atlantic (IT)', 'France 5', 'W9',
       'Antena 3', 'tvk', 'TV Aichi', 'PBS', 'Hunan TV', 'TV3 (ES)',
       'ReelzChannel', 'Nickelodeon', 

On met le df transformé sous format csv

In [27]:
series_info4.to_csv('C:/Users/Jean-Baptiste/OneDrive/ENSAE/projet_python_data_final.csv')